# NLP Starter — Colab/Kaggle Quickstart

> Author : Badr TAJINI

**Academic year:** 2025–2026  
**School:** ECE  
**Course:** Machine Learning & Deep Learning 2 

---

Welcome! This notebook walks you through the AG News text-classification starter on a free GPU runtime. Follow the steps in order to check the environment, install dependencies, and optionally train/evaluate the LSTM model.

## 0. Project files

- Option A: `git clone <your_repo_url> nlp-project`
- Option B: Upload the `nlp-project` folder via the sidebar (it should appear as `/content/nlp-project` in Colab).

Run the next cells afterward; they will raise helpful errors if the folder is missing.

### Quick checklist before running code
- Switch the runtime to **GPU (T4)** first.
  - Colab: `Runtime → Change runtime type → GPU → Save`.
  - Kaggle: gear icon → enable **Accelerator**, choose `T4 x1`.
- Wait for the runtime to reconnect.
- Run each cell from top to bottom. If a cell errors, fix the issue, then rerun that same cell.

### How to run a cell
- Click the ▶️ button on the left of a cell, or press **Shift+Enter** (Colab) / **Ctrl+Enter** (Kaggle).
- A cell is finished when a number like `[1]` appears on the left.
- Do not skip cells; the later ones depend on the setup above them.

### Step 0 — Confirm the GPU is ready
Run the next cell. You should see a table with GPU details (name + memory). If you see `nvidia-smi unavailable`, the runtime is still on CPU—go back and switch it to GPU, then rerun this cell.

In [1]:
!nvidia-smi || echo "nvidia-smi unavailable (CPU runtime)"


Fri Nov 14 21:18:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8              2W /   50W |    2839MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Step 1 — Point the notebook at the project folder
This cell makes sure the notebook is executing inside the `nlp-project` directory.
If it raises a `FileNotFoundError`, double-check where you uploaded/cloned the folder, adjust the path, and rerun.

In [2]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
elif PROJECT_ROOT.name == "content":
    candidate = PROJECT_ROOT / "nlp-project"
    if candidate.exists():
        PROJECT_ROOT = candidate.resolve()

if not (PROJECT_ROOT / "src").exists():
    raise FileNotFoundError(
        f"Could not locate project root at {PROJECT_ROOT}. Upload or clone nlp-project before proceeding."
    )

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))
print(f"Project root: {PROJECT_ROOT}")


Project root: C:\Users\lucas\End-to-End-Deep-Learning-Systems\End-to-End-Deep-Learning-Systems\starters\nlp-project-starter\nlp-project


### Step 2 — Install the project requirements
This command reads `requirements.txt` and installs the exact package versions used locally. Expect a lot of output; that's normal. If installation fails, run the cell again before moving on.

In [3]:
# Install project dependencies listed in requirements.txt
!pip install -r requirements.txt


### Step 3 — Run the smoke test
This quick check downloads AG News (first run only), builds the vocabulary, and runs one mini-batch through the LSTM. It saves `outputs/smoke_metrics.json` so you know the pipeline works.
If the cell reports a network/download issue, wait a few seconds and rerun it.

In [4]:
from src import smoke_check

smoke_path = smoke_check.run_smoke("configs/nlp_agnews.yaml")
print(smoke_path.read_text())


AttributeError: Can't pickle local object 'build_loaders.<locals>.<lambda>'

## 1. Review smoke-test output
- Confirm the previous cell printed a JSON block (loss, batch size, seq_len).
- You should now see `outputs/smoke_metrics.json` in the file browser on the left.
- Only need a quick check? You can stop here. Ready for full training? Continue to Section 2.
- If anything failed, read the error message, fix the issue, and rerun the smoke cell before moving on.

## 2. Full training run (optional)
Only run these cells when you want the complete experiment. On the first run, downloading the dataset and building the vocab may take a couple of minutes.

**Before running:**
1. Open `configs/nlp_agnews.yaml` (File → Open) if you want to tweak hyperparameters (epochs, learning rate, etc.).
2. Ensure the runtime still shows a GPU connection.
3. Close other heavy browser tabs to free memory.

In [ ]:
# Train the model using the main configuration file. Expect visible progress bars.
# The first epochs may start slowly while the dataset finishes downloading.
!python src/train.py --config configs/nlp_agnews.yaml



train:   0%|          | 0/1688 [00:00<?, ?it/s]
                                               

-------------------------------------------------------------------------------
train.py 101 <module>
main(args.config)

train.py 78 main
tr_loss = train_one_epoch(model, train_loader, crit, opt, device)

train.py 17 train_one_epoch
for toks, lens, y in tqdm(loader, desc="train", leave=False):

std.py 1181 __iter__
for obj in iterable:

dataloader.py 484 __iter__
return self._get_iterator()

dataloader.py 415 _get_iterator
return _MultiProcessingDataLoaderIter(self)

dataloader.py 1138 __init__
w.start()

process.py 121 start
self._popen = self._Popen(self)

context.py 224 _Popen
return _default_context.get_context().Process._Popen(process_obj)

context.py 336 _Popen
return Popen(process_obj)

popen_spawn_win32.py 93 __init__
reduction.dump(process_obj, to_child)

reduction.py 60 dump
ForkingPickler(file, protocol).dump(obj)

AttributeError:
Can't pickle local object 'build_loaders.<locals

In [ ]:
# Evaluate the best checkpoint produced during training.
# This prints validation/test metrics and writes eval.json to the outputs folder.
!python src/evaluate.py --config configs/nlp_agnews.yaml --ckpt outputs/best.pt


C:\Users\lucas\End-to-End-Deep-Learning-Systems\End-to-End-Deep-Learning-Systems\starters\nlp-project-starter\nlp-project\src\evaluate.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen

### Step 4 — What should I see now?
- `outputs/best.pt`: saved checkpoint with the trained weights and vocabulary.
- `outputs/log.csv`: training history (loss/accuracy/F1 per epoch).
- `outputs/metrics.json`: summary of the best validation F1.
- `outputs/eval.json`: validation (and test, if available) accuracy/F1.
If any of these files are missing, scroll up to check for errors in the training/evaluation cells.

## 3. Mirror this workflow for other runs
1. Duplicate this notebook and rename it for your custom dataset (e.g., `01_custom_dataset.ipynb`).
2. Edit the install cell if you add new packages to `requirements.txt`.
3. Replace the config path with your own YAML file (CSV dataset, different splits, etc.).
4. Use `python src/predict.py --ckpt outputs/best.pt --texts texts.json` to score custom sentences once training finishes.

Following the same structure—GPU check → install → smoke test → full run—keeps your experiments tidy and reproducible.